### Preliminaries

In [9]:
import os
HOME = os.environ["HOME"]
CARDIAC_GWAS_REPO = f"{HOME}/01_repos/CardiacGWAS"
CARDIAC_COMA_REPO = f"{HOME}/01_repos/CardiacCOMA/"
MLRUNS_DIR = f"{CARDIAC_COMA_REPO}/mlruns"
os.chdir(CARDIAC_COMA_REPO)

In [10]:
import mlflow
from mlflow.tracking import MlflowClient

import os, sys

import torch
import torch.nn.functional as F

from config.cli_args import overwrite_config_items
from config.load_config import load_yaml_config, to_dict

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image

import pandas as pd
import shlex
from subprocess import check_output

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

# import model.Model3D
# from utils.helpers import get_coma_args, get_lightning_module, get_datamodule
from copy import deepcopy
from pprint import pprint

from typing import List
from tqdm import tqdm

import pandas as pd

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

In [11]:
os.chdir(CARDIAC_GWAS_REPO)

import glob
import os
import numpy as np
import pandas as pd
from IPython import embed
import functools

from data.gwas_details import GWAS_dict
GWAS_PATH = "data/other_gwas/original_files"

In [12]:
gwas_harmonized_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}.tsv"
gwas_selected_snps_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}__selected_snps.tsv"
COMA_GWAS_SUMMARY = "results/gwas_loci_summary_across_runs.csv"
LOGP_PATH = "results/log10p_for_selected_snps_across_gwas.csv"

### GWAS file - reduction

In [13]:
for file, info in GWAS_dict.items():
    prefix = info["prefix"]
    phenotype = info["phenotype"].replace(" ", "_")
    o_filename = f"{prefix}__{phenotype}.tsv"
    file = os.path.join(GWAS_PATH, file)
    df = pd.read_csv(file, sep="\t").rename(info["columns"], axis=1)    
    df = df[["CHR", "BP", "SNP", "P"]]
    df.to_csv(o_filename, sep='\t', header=True, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/other_gwas/original_files/Aung2019/AungN_31554410_LVMVR.txt'

___

### Filter GWAS files for specific SNPS

#### Find significant SNPs across COMA runs.

In [14]:
gwas_loci_summary_across_runs_df

,run,pheno,region,CHR,SNP,BP,AF,a_0,a_1,BETA,SE,T,P,locus_name
0,0285fa2356fd454e88e3c30d6b63f163,1_0285f_z007,chr6_78,6.0,rs11153730,118667522.0,0.49203,T,C,-0.076241,0.007928,-9.6162,7.311391e-22,PLN
1,0285fa2356fd454e88e3c30d6b63f163,1_0285f_z007,chr6_79,6.0,rs10872167,118988362.0,0.46056,A,G,-0.071901,0.007982,-9.0084,2.202926e-19,PLN
2,0285fa2356fd454e88e3c30d6b63f163,1_0285f_z012,chr2_108,2.0,rs2042995,179558366.0,0.22166,T,C,-0.071122,0.009542,-7.4534,9.332543e-14,TTN
3,0285fa2356fd454e88e3c30d6b63f163,1_0285f_z004,chr2_108,2.0,rs2042995,179558366.0,0.22166,T,C,0.069439,0.009544,7.2755,3.531832e-13,TTN
4,0285fa2356fd454e88e3c30d6b63f163,1_0285f_z006,chr2_108,2.0,rs2042995,179558366.0,0.22166,T,C,-0.069385,0.009546,-7.2685,3.723917e-13,TTN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,ff7c594accb74ad7947621a6b3e2527a,1_ff7c5_z002,chr6_79,6.0,rs11756440,118993642.0,0.47505,C,A,0.046588,0.007990,5.8307,5.573141e-09,PLN
1202,ff7c594accb74ad7947621a6b3e2527a,1_ff7c5_z011,chr17_27,17.0,rs117953218,45244074.0,0.13850,T,C,-0.067495,0.011645,-5.7959,6.861199e-09,GOSR2
1203,ff7c594accb74ad7947621a6b3e2527a,1_ff7c5_z007,chr17_27,17.0,rs11570508,45228560.0,0.22405,C,A,-0.055592,0.009600,-5.7911,7.059924e-09,GOSR2
1204,ff7c594accb74ad7947621a6b3e2527a,1_ff7c5_z006,chr6_79,6.0,rs11756440,118993642.0,0.47505,C,A,-0.045982,0.007994,-5.7521,8.892011e-09,PLN


In [15]:
gwas_loci_summary_across_runs_df = pd.read_csv(COMA_GWAS_SUMMARY)

# get index of best locus/variable
idx = gwas_loci_summary_across_runs_df.groupby(["region"])["P"].transform(min) == gwas_loci_summary_across_runs_df["P"]

best_association_per_region = gwas_loci_summary_across_runs_df[idx].sort_values("region")
best_snps = set(best_association_per_region.SNP)

In [16]:
best_association_per_region

,run,pheno,region,CHR,SNP,BP,AF,a_0,a_1,BETA,SE,T,P,locus_name
1012,e6490cf8389f4e7da52fef013562770f,1_e6490_z012,chr10_20,10.0,rs2797593,28655648.0,0.358680,A,C,0.049402,0.008336,5.9265,3.126079e-09,Unnamed
593,8b630b3a67f94ec2853645fe0314a424,1_8b630_z003,chr10_69,10.0,rs189569984,112544125.0,0.008998,C,T,-0.246570,0.041965,-5.8757,4.253045e-09,RBM20
815,b0ca240bf966455888005eee1e1fed99,1_b0ca2_z004,chr10_74,10.0,rs375034445,121424815.0,0.214880,A,AT,-0.056170,0.009752,-5.7600,8.489850e-09,Unnamed
744,a7d3200ce0fb43d586d53db5afff50d9,1_a7d32_z005,chr11_2,11.0,rs569550,1887068.0,0.386020,T,G,-0.057778,0.008179,-7.0643,1.648162e-12,LSP1*
889,cc438c4acf664d5ab05ed79686968a06,1_cc438_z003,chr11_32,11.0,rs371701829,57864175.0,0.321240,CT,C,0.053399,0.008821,6.0537,1.431858e-09,Unnamed
855,b6a7eedfc0e84b8b9d1f099ed5f158c4,1_b6a7e_z009,chr11_37,11.0,rs7119588,66377035.0,0.057960,T,C,-0.109830,0.017633,-6.2285,4.767602e-10,Unnamed
458,64bde9a562c24419b0cc96b1e922ec95,1_64bde_z000,chr12_19,12.0,rs3741760,28544464.0,0.755160,G,A,0.066336,0.009250,7.1713,7.585776e-13,CCDC91*
653,9976e04864744db58cabdef4a6c473fc,1_9976e_z001,chr12_47,12.0,rs764004172,78199435.0,0.431320,TAGTA,T,-0.049736,0.008058,-6.1722,6.813967e-10,Unnamed
346,4b7658276c0c4f12a75d95f67d7f3aa8,1_4b765_z000,chr12_67,12.0,rs35350651,111907431.0,0.505280,A,AC,0.053989,0.007948,6.7927,1.119438e-11,Unnamed
515,7382663c96c44c00a5bb6d119776c01b,1_73826_z000,chr12_69,12.0,rs4767239,114816548.0,0.825660,G,C,0.073146,0.010610,6.8942,5.520774e-12,TBX5


In [17]:
os.chdir(CARDIAC_COMA_REPO)

In [18]:
regions = { 
    f"{assoc[1].region} ({assoc[1].P:.1e})": assoc[1].region 
    for assoc in best_association_per_region.sort_values("P").iterrows() 
} 

del regions["chr6_79 (4.5e-20)"]

@interact
def manhattan(
    region=widgets.Select(options=regions, description="Locus: \n",),
    PC_adjusted=False
):
    
    from PIL import Image
    best_association_per_region
    _best_association_per_region = best_association_per_region.set_index("region")
    
    assoc = _best_association_per_region.loc[region]
    
    run_id, z_variable = assoc.run, assoc.pheno[-4:]
    
    if PC_adjusted: 
        manhattan_file = f"mlruns/1/{run_id}/artifacts/GWAS_adj_10PCs/figures/GWAS__{z_variable}__1_{run_id}__manhattan.png"
        qq_file = f"mlruns/1/{run_id}/artifacts/GWAS_adj_10PCs/figures/GWAS__{z_variable}__1_{run_id}__QQ-plot.png"
    else:
        manhattan_file = f"mlruns/1/{run_id}/artifacts/GWAS/figures/{z_variable}__manhattan.png"
        qq_file = f"mlruns/1/{run_id}/artifacts/GWAS/figures/{z_variable}__QQ-plot.png"

    print(run_id, z_variable)
    display(Image.open(manhattan_file))
    display(Image.open(qq_file))

interactive(children=(Select(description='Locus: \n', options={'chr6_78 (1.8e-22)': 'chr6_78', 'chr17_27 (3.9e…

#### Filter GWAS files for selected SNPs

In [17]:
for file, info in GWAS_dict.items():
    
    prefix = info["prefix"]
    phenotype = info["phenotype"].replace(" ", "_")
    
    o_filename = gwas_harmonized_pattern.format(prefix=prefix, phenotype=phenotype)           
    ofile_selected = gwas_selected_snps_pattern.format(prefix=prefix, phenotype=phenotype)           
    
    df = pd.read_csv(o_filename, sep="\t")
    keep = df.apply(lambda row: row.SNP in best_snps, axis=1)    
    
    print(ofile_selected)
    df[keep].to_csv(ofile_selected, sep="\t", index=False, header=True)        

KeyboardInterrupt: 

In [14]:
pp = []

for file, info in GWAS_dict.items():
    
    prefix = info["prefix"]
    phenotype = info["phenotype"].replace(" ", "_")
        
    ofile_selected_snps = gwas_selected_snps_pattern.format(prefix=prefix, phenotype=phenotype)
    
    df = pd.read_csv(ofile_selected, sep="\t")
    df = df[["SNP", "P"]]
    df = df.rename({"P": f"{prefix}__{phenotype}"}, axis=1)
    
    pp.append(df)
    

pp = [dd.set_index("SNP") for dd in pp]
snps_across_gwas = functools.reduce(lambda df1,df2: pd.merge(df1,df2,on='SNP'), pp)

# WHAT'S HAPPENING WITH THIS SNP?
snps_across_gwas = snps_across_gwas.drop("rs533885")
log10p_gwas_df = (-np.log10(snps_across_gwas))
log10p_gwas_df.to_csv(LOGP_PATH, index=True)

In [21]:
best_association_per_region

,run,pheno,region,CHR,SNP,BP,AF,a_0,a_1,BETA,SE,T,P,locus_name
1012,e6490cf8389f4e7da52fef013562770f,1_e6490_z012,chr10_20,10.0,rs2797593,28655648.0,0.358680,A,C,0.049402,0.008336,5.9265,3.126079e-09,Unnamed
593,8b630b3a67f94ec2853645fe0314a424,1_8b630_z003,chr10_69,10.0,rs189569984,112544125.0,0.008998,C,T,-0.246570,0.041965,-5.8757,4.253045e-09,RBM20
815,b0ca240bf966455888005eee1e1fed99,1_b0ca2_z004,chr10_74,10.0,rs375034445,121424815.0,0.214880,A,AT,-0.056170,0.009752,-5.7600,8.489850e-09,Unnamed
744,a7d3200ce0fb43d586d53db5afff50d9,1_a7d32_z005,chr11_2,11.0,rs569550,1887068.0,0.386020,T,G,-0.057778,0.008179,-7.0643,1.648162e-12,LSP1*
889,cc438c4acf664d5ab05ed79686968a06,1_cc438_z003,chr11_32,11.0,rs371701829,57864175.0,0.321240,CT,C,0.053399,0.008821,6.0537,1.431858e-09,Unnamed
855,b6a7eedfc0e84b8b9d1f099ed5f158c4,1_b6a7e_z009,chr11_37,11.0,rs7119588,66377035.0,0.057960,T,C,-0.109830,0.017633,-6.2285,4.767602e-10,Unnamed
458,64bde9a562c24419b0cc96b1e922ec95,1_64bde_z000,chr12_19,12.0,rs3741760,28544464.0,0.755160,G,A,0.066336,0.009250,7.1713,7.585776e-13,CCDC91*
653,9976e04864744db58cabdef4a6c473fc,1_9976e_z001,chr12_47,12.0,rs764004172,78199435.0,0.431320,TAGTA,T,-0.049736,0.008058,-6.1722,6.813967e-10,Unnamed
346,4b7658276c0c4f12a75d95f67d7f3aa8,1_4b765_z000,chr12_67,12.0,rs35350651,111907431.0,0.505280,A,AC,0.053989,0.007948,6.7927,1.119438e-11,Unnamed
515,7382663c96c44c00a5bb6d119776c01b,1_73826_z000,chr12_69,12.0,rs4767239,114816548.0,0.825660,G,C,0.073146,0.010610,6.8942,5.520774e-12,TBX5


In [47]:
logp = pd.DataFrame(best_association_per_region.apply(lambda row: -np.log10(row.P), axis=1)).rename({0: '-log10(p)'}, axis=1)

In [48]:
logp

,-log10(p)
1012,8.5050
593,8.3713
815,8.0711
744,11.7830
889,8.8441
855,9.3217
458,12.1200
653,9.1666
346,10.9510
515,11.2580


In [54]:
z_assoc = pd.concat([best_association_per_region[["SNP", "region", "locus_name"]], logp], axis=1)

In [59]:
snps_across_gwas

,NAY2019__LVMVR,NAY2019__LVEDV,NAY2019__LVM,NAY2019__LVEF,NAY2019__LVESV,PIRRUCCELLO2020__LVEDV,PIRRUCCELLO2020__LVEDVi,PIRRUCCELLO2020__LVESV,PIRRUCCELLO2020__LVESVi,PIRRUCCELLO2020__LVEF,...,MEYER2020__LV_fractal_dim_slice9,MEYER2020__LV_fractal_dim_slice1,MEYER2020__LV_fractal_dim_slice3,MEYER2020__LV_fractal_dim_slice6,PIRRUCCELLO2022__Ascending_thoracic_aortic_diameter,PIRRUCCELLO2022__Descending_thoracic_aortic_diameter,MyGWAS__LVEDV,MyGWAS__LVSph,MyGWAS__LVM,MyGWAS__RVEDV
SNP,,,,,,,,,,,,,,,,,,,,,
rs12744578,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,...,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02,1.400000e-02
rs35001652,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,...,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01,6.600000e-01
1:100033312_TA_T,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,...,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02,8.700000e-02
rs3806234,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,...,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01,1.200000e-01
1:155222001_CT_C,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,...,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01,7.700000e-01
rs823096,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,...,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02,1.200000e-02
1:219699007_ACT_A,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,...,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01,9.600000e-01
rs2853621,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,...,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01,1.500000e-01
rs2245109,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,...,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01,2.100000e-01


,NAY2019__LVMVR,NAY2019__LVEDV,NAY2019__LVM,NAY2019__LVEF,NAY2019__LVESV,PIRRUCCELLO2020__LVEDV,PIRRUCCELLO2020__LVEDVi,PIRRUCCELLO2020__LVESV,PIRRUCCELLO2020__LVESVi,PIRRUCCELLO2020__LVEF,...,PIRRUCCELLO2022__Ascending_thoracic_aortic_diameter,PIRRUCCELLO2022__Descending_thoracic_aortic_diameter,MyGWAS__LVEDV,MyGWAS__LVSph,MyGWAS__LVM,MyGWAS__RVEDV,SNP,region,locus_name,-log10(p)
877,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,...,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,-4.214670,rs11153730,chr6_78,PLN,21.7460
457,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,...,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,-2.619789,rs10872167,chr6_79,PLN,19.3450
295,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,...,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,-3.275724,rs17608766,chr17_27,GOSR2,15.4060
1014,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,...,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,-10.638272,rs2042995,chr2_108,TTN,13.2740
458,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,...,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,-0.292430,rs3741760,chr12_19,CCDC91*,12.1200
744,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,...,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,-4.142668,rs569550,chr11_2,LSP1*,11.7830
515,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,...,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,-1.568636,rs4767239,chr12_69,TBX5,11.2580
77,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,...,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,-1.721246,rs66921363,chr21_10,NCSTNP1*,11.0590
346,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,...,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,-7.698970,rs35350651,chr12_67,Unnamed,10.9510
577,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,...,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,-0.086186,rs3094087,chr6_25,Unnamed,10.7840


In [62]:
log10p_gwas_df = pd.read_csv(LOGP_PATH)
log10p_gwas_df

,SNP,NAY2019__LVMVR,NAY2019__LVEDV,NAY2019__LVM,NAY2019__LVEF,NAY2019__LVESV,PIRRUCCELLO2020__LVEDV,PIRRUCCELLO2020__LVEDVi,PIRRUCCELLO2020__LVESV,PIRRUCCELLO2020__LVESVi,...,MEYER2020__LV_fractal_dim_slice9,MEYER2020__LV_fractal_dim_slice1,MEYER2020__LV_fractal_dim_slice3,MEYER2020__LV_fractal_dim_slice6,PIRRUCCELLO2022__Ascending_thoracic_aortic_diameter,PIRRUCCELLO2022__Descending_thoracic_aortic_diameter,MyGWAS__LVEDV,MyGWAS__LVSph,MyGWAS__LVM,MyGWAS__RVEDV
0,rs10872167,0.522879,2.619789,1.698970,0.468521,2.000000,11.657577,7.795880,7.113509,5.508638,...,0.532830,0.270330,0.196550,10.062000,0.795880,1.853872,9.914500,13.08600,5.88580,9.28220
1,rs11153730,0.853872,4.214670,2.292430,0.568636,3.000000,15.431798,10.823909,9.167491,7.267606,...,0.922240,0.339850,0.003448,10.439000,1.000000,1.853872,10.916000,14.82300,5.49730,9.38830
2,rs114563403,0.958607,0.283997,1.585027,0.420216,0.013228,1.000000,0.221849,0.886057,0.431798,...,0.254370,0.498140,0.152140,0.961310,1.173925,6.468521,0.333360,0.62383,1.95340,2.43710
3,rs11706187,0.431798,2.004365,1.677781,0.468521,1.698970,3.387216,3.522879,2.823909,2.853872,...,0.497840,1.028200,0.258610,0.142670,0.376751,1.060481,3.978400,4.70400,4.31790,5.21970
4,rs12206258,1.107905,2.173925,0.585027,0.040959,1.408935,2.795880,3.397940,1.795880,2.060481,...,0.996300,0.111100,0.544440,0.183450,0.721246,1.113509,6.265400,0.10874,2.30880,5.27510
5,rs12744578,2.000000,1.853872,0.214670,6.309804,5.468521,8.026872,7.886057,17.259637,18.568636,...,0.164290,0.053807,0.922860,0.205190,1.070581,0.508638,3.628000,2.47470,2.47130,2.85750
6,rs139078271,2.585027,-0.000000,2.769551,0.602060,0.251812,3.070581,0.244125,1.522879,0.154902,...,0.031424,0.524880,1.131600,1.097100,3.886057,1.721246,3.103500,0.80591,6.82790,3.45840
7,rs141412536,0.537602,0.468521,1.214670,0.337242,0.468521,2.050610,2.698970,1.638272,2.045757,...,0.389680,0.045088,1.455000,0.192950,0.070581,0.187087,6.107000,0.93818,5.46430,5.33340
8,rs162748,0.366532,0.229148,0.136677,0.327902,0.443697,0.102373,0.236572,0.004365,0.017729,...,0.325200,0.067881,0.000808,1.395900,0.721246,0.494850,0.198080,8.72720,0.40826,0.13539
9,rs17598658,1.619789,0.148742,1.721246,0.309804,0.366532,0.214670,0.075721,0.055517,0.075721,...,0.580190,2.985000,0.351720,2.145800,3.356547,0.920819,0.081898,0.87702,0.45438,0.51391


In [84]:
assoc_df = pd.merge(z_assoc, log10p_gwas_df.set_index("SNP"), right_index=True, left_on="SNP").sort_values(by="-log10(p)", ascending=False)
assoc_df = assoc_df.set_index("SNP").drop("rs10872167").reset_index()
assoc_df = assoc_df.set_index(assoc_df.locus_name.where(assoc_df.locus_name != "Unnamed", assoc_df.region))

In [87]:
assoc_df.to_csv("results/log_p_assoc.csv")